# Download datasets

Slednava kelija sodrzhi kod koj go najdov na internet za simnuvanje na fajlovi od javni linkovi od google drive, bidejkji za datasetite ne sakashe da raboti gdown.

In [ ]:
import requests

def download_file_from_google_drive(URL, destination):
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

    id = URL.split('=')[1]

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

In [ ]:
!mkdir /content/datasets/

mkdir: cannot create directory ‘/content/datasets/’: File exists


In [ ]:
# NOTE: This is the sampled version of nlu-nt, not the full dataset
download_file_from_google_drive('https://drive.google.com/uc?id=1JYk14D7wUaqQoUZLXS55nJXYbFStyr1T', '/content/datasets/nlu_nt_train.tsv')
download_file_from_google_drive('https://drive.google.com/uc?id=1Q7nhvmNTMa8okzouQbylJLnxKwgAv-EH', '/content/datasets/nlu_nt_test.tsv')

download_file_from_google_drive('https://drive.google.com/uc?id=1QjG7O_APcnzmnziqnsGjEHm_K0MH1vNh', '/content/datasets/msr_paraphrase_train.txt')
download_file_from_google_drive('https://drive.google.com/uc?id=1GRV4-ctCsgeyYUnzW4gn15US5UUZctQU', '/content/datasets/msr_paraphrase_test.txt')

download_file_from_google_drive('https://drive.google.com/uc?id=14sEN3h8ZK10WxPIzYx7YWRhWrWKqR0kY', '/content/datasets/paws_wiki_train.tsv')
download_file_from_google_drive('https://drive.google.com/uc?id=1zK6EYw68wQmf9PuoYuphKHWS3m9nRPLb', '/content/datasets/paws_wiki_test.tsv')

download_file_from_google_drive('https://drive.google.com/uc?id=1-EWR9auxBOWIOzo1y6PlYB1hCS8ihnfr', '/content/datasets/combined_dataset_train.csv')
download_file_from_google_drive('https://drive.google.com/uc?id=1-92vLJxYm8PK73vKNmlKmsSa1p9w78nB', '/content/datasets/combined_dataset_test.csv')

# Functions and setup

In [ ]:
!pip install comet_ml transformers tensorflow-text sacrebleu bert_score rouge_score datasets sentencepiece deep-translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import torch
from comet_ml import Experiment
from tqdm.auto import tqdm
from datasets import load_metric

import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text as text  # Needed for loading universal-sentence-encoder-cmlm/multilingual-preprocess
import numpy as np

from transformers import BartForConditionalGeneration, BartTokenizer, get_cosine_schedule_with_warmup, get_constant_schedule_with_warmup
from transformers.models.bart.modeling_bart import shift_tokens_right
from transformers import T5ForConditionalGeneration, T5Tokenizer, PegasusForConditionalGeneration, PegasusTokenizer, GPT2Tokenizer, GPTNeoForCausalLM

import pickle

Ovaa funkcija go chita datasetot msr_paraphrase.

In [ ]:
def read_dataset1(path):
  data = []

  with open(path) as file:
    for idx, line in enumerate(file):
      if idx == 0:
        continue

      tokens = line.split("\t")

      if tokens[0] == '1':
        data.append({'Quality': tokens[0], '#1 ID': tokens[1], '#2 ID':tokens[2], '#1 String': tokens[3], "#2 String": tokens[4]})

  return pd.DataFrame(data)

Ovaa funkcija go chita datasetot paws.

In [ ]:
def read_dataset_paws(path):
  dataframe = pd.read_csv(path, sep='\t', index_col=0)
  dataframe.rename(columns={'sentence1': "#1 String", 'sentence2': '#2 String'}, inplace=True)
  dataframe = dataframe[dataframe['label'] == 1]
  return dataframe[['#1 String', '#2 String']]

Ovaa funkcija go chita datasetot nlu nt.

In [ ]:
def read_dataset_nlu_nt(path):
  dataframe = pd.read_csv(path, sep='\t')
  dataframe.rename(columns={'input_text': "#1 String", 'target_text': "#2 String"}, inplace=True)
  return dataframe[['#1 String', '#2 String']]

Ovaa funkcija go chita primerokot od datasetot nlu nt shto go koristam za treniranje i evaluiranje na modelite bidejkji celosniot dataset beshe pregolem. Mnogu slichna so prethodnata f-ja samo shto poinaku se krsteni kolonite vo primerokot, pa nema potreba da im pravam rename.

In [ ]:
def read_dataset_nlu_nt_sample(path):
  dataframe = pd.read_csv(path, sep='\t')
  return dataframe[['#1 String', '#2 String']]

In [ ]:
def read_combined_dataset(path):
  dataframe = pd.read_csv(path)
  return dataframe[['#1 String', '#2 String']]

Ovaa f-ja dodava "summarize:" na pochetokot od sekoja vlezna rechenica od podatochnoto mnozhestvo, bidejkji takov input bara T5. Ova ne e problem, iako trenirame za parafraziranje

In [ ]:
def adapt_dataset_for_t5(dataframe):
  dataframe['#1 String'] = dataframe['#1 String'].map(lambda x: f"paraphrase: {x}")

Vamsi/T5 bara na pochetokot "paraphrase:" oti e pretreniran za parafraziranje, pa zatoa vaka. Ovoj model ne go koristam vo finalnite eksperimenti.

In [ ]:
def adapt_dataset_for_t5_vamsi(dataframe):
  dataframe['#1 String'] = dataframe['#1 String'].map(lambda x: f"paraphrase: {x}")

Ovaa f-ja go zema datasetot kako vlez i go priprema na soodveten nachin za da mozhe da se koristi so modelite, vrakja rechnik so site potrebni podatoci.

In [ ]:
def preprocess_for_gpt_neo(dataframe):
  dataframe['train'] = dataframe['#1 String'] + ' Paraphrase: ' + dataframe['#2 String']
  dataframe['test'] = dataframe['#1 String'] + ' Paraphrase: '

In [ ]:
def prepare_and_tokenize_dataset_gpt_neo(raw_dataframe, tokenizer, model):
  tokenizer.pad_token = -100

  data_x = tokenizer(raw_dataframe['train'].tolist(), padding='max_length',
                     max_length=raw_dataframe['train'].map(lambda x: len(x)).max(), return_tensors='pt')

  data_x['labels'] = data_x['input_ids'].clone()
  data_x['labels'][data_x['attention_mask'] == 0] = -100

  return data_x

In [ ]:
def prepare_and_tokenize_test_dataset_gpt_neo(raw_dataframe, tokenizer):
  tokenizer.pad_token = -100

  data_x = tokenizer(raw_dataframe['test'].tolist(), padding='max_length',
                     max_length=raw_dataframe['test'].map(lambda x: len(x)).max(), return_tensors='pt')
  
  with tokenizer.as_target_tokenizer():
      labels = tokenizer(raw_dataframe['#2 String'].tolist(), add_special_tokens=False)

  return data_x, labels['input_ids'], raw_dataframe['#2 String'].tolist(), raw_dataframe['#1 String'].tolist()

In [ ]:
def tokenize_and_prepare_dataset(raw_dataframe, tokenizer, model):
  data_x = tokenizer(raw_dataframe['#1 String'].tolist(), padding='max_length',
                     max_length=raw_dataframe['#1 String'].map(lambda x: len(x)).max(), return_tensors='pt')
  
  data_y = None

  with tokenizer.as_target_tokenizer():
      data_y = tokenizer(raw_dataframe['#2 String'].tolist(), padding='max_length',
                              max_length=raw_dataframe['#2 String'].map(lambda x: len(x)).max(), return_tensors='np')
  data_y = data_y['input_ids']
  data_y[data_y == model.config.pad_token_id] = -100

  data_x['labels'] = torch.tensor(data_y)

  data_x['decoder_input_ids'] = shift_tokens_right(data_x['labels'], model.config.pad_token_id, model.config.decoder_start_token_id)

  return data_x

Ovaa f-ja e mnogu slichna na prethodnata, so razlika shto za testirachkiot dataset se potrebni drugi informacii pa niv gi vrakja.

In [ ]:
def tokenize_and_prepare_test_dataset(raw_dataframe, tokenizer):
  data_x = tokenizer(raw_dataframe['#1 String'].tolist(), padding='max_length',
                     max_length=raw_dataframe['#1 String'].map(lambda x: len(x)).max(), return_tensors='pt')

  data_y = None
  with tokenizer.as_target_tokenizer():
      data_y = tokenizer(raw_dataframe['#2 String'].tolist(), add_special_tokens=False)
      
  return data_x, data_y['input_ids'], raw_dataframe['#2 String'].tolist(), raw_dataframe['#1 String'].tolist() # data, tokenized_labels, untokenized_labels, untokenized_input

Klasa koja definira dataset koj proizleguva od nekoj rechnik.

In [ ]:
class DatasetFromDictData(torch.utils.data.Dataset):
  def __init__(self, data, transform=None, target_transform=None):
    self.data = data
    self.transform = transform
    self.target_transform = target_transform

  def __len__(self):
    if 'input_ids' in self.data:
      return len(self.data['input_ids'])
    else:
      return len(self.data[list(self.data.keys())[0]])

  def __getitem__(self, idx):
    return {key: self.data[key][idx] for key in self.data.keys()}

Ovaa f-ja gi priprema site raboti za treniranjeto, i go trenira modelot, pritoa loggirajkji gi podatocite na comet. Isto taka imam implementirano i mozhnost za validacija so validacisko mnozhestvo, no toa ne go koristam bidejkji odzema premnogu vreme i go pravi treniranjeto predolgo 

In [ ]:
def train(model, dataloader, experiment, hyperparameters, tokenized_labels, do_logging=True, validation_dataloader=None):

  optimizer = torch.optim.AdamW(model.parameters(), lr=hyperparameters['lr_rate'])# lr=0.0005)

  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  model.to(device)

  num_training_steps = hyperparameters['num_epochs'] * len(dataloader)

  #lr_scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=int(0.05*num_training_steps), num_training_steps=num_training_steps)
  lr_scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=int(hyperparameters['num_warmup_steps_percent_of_all']*num_training_steps))

  progress_bar = tqdm(range(num_training_steps))

  global_step = 0

  experiment.log_parameters(hyperparameters)

  model.train()
  for epoch in range(hyperparameters['num_epochs']):
    running_avg = 0.0
    running_avg_loss = 0.0
    epoch_step = 0

    if epoch == 0 and validation_dataloader is not None:
      # do a validation run before training
      rouge_score, sacrebleu_score, bertscore_score, labse_score, _ = evaluate(model, validation_dataloader, device, None, untokenized_labels)

      to_log = get_metrics_dict(rouge_score, sacrebleu_score, bertscore_score)
      to_log['labse_score'] = labse_score
      
      experiment.log_metrics(to_log, epoch=epoch)

    for batch in dataloader:
      batch = {k: v.to(device) for k, v in batch.items()}

      outputs = model(**batch)
      loss = outputs.loss
      loss.backward()

      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()

      progress_bar.update(1)

      if not do_logging:
        del batch
        continue

      preds = torch.argmax(outputs.logits.detach(), dim=-1)
      labels_np = batch['labels'].detach()

      num_correct = (labels_np[labels_np != -100] == preds[labels_np != -100]).sum().item()
      num_total = labels_np[labels_np != -100].numel()

      del batch
      del preds
      del labels_np

      #num_correct = (preds == batch['labels'] and batch['labels'] != -100).sum().item()
      #num_total = batch['labels'][batch['labels'] != -100].numel()

      batch_acc = (float(num_correct)/num_total)
      
      epoch_step += 1
      global_step += 1

      the_loss = loss.item()
      
      running_avg = ((epoch_step-1)/float(epoch_step))*running_avg + batch_acc/float(epoch_step)
      running_avg_loss = ((epoch_step-1)/float(epoch_step))*running_avg_loss + the_loss/float(epoch_step)

      progress_bar.set_postfix_str(f"batch_loss={the_loss:.5f} epoch_running_avg_loss={running_avg_loss:.5f} batch_acc={batch_acc:.5f}\tepoch_running_avg_acc={running_avg:.5f}", refresh=True)

      experiment.log_metric('batch_acc', batch_acc, step=global_step)
      experiment.log_metric(f'batch_acc_epoch_{epoch}', batch_acc, step=epoch_step)

      experiment.log_metric('batch_loss', the_loss, step=global_step)
      
      experiment.log_metric('per_epoch_running_avg_acc', running_avg, step=global_step)
      experiment.log_metric('per_epoch_running_avg_loss', running_avg_loss, step=global_step)
      
    if validation_dataloader is not None:
      rouge_score, sacrebleu_score, bertscore_score, labse_score, _ = evaluate(model, validation_dataloader, device, None, untokenized_labels)

      to_log = get_metrics_dict(rouge_score, sacrebleu_score, bertscore_score)
      to_log['labse_score'] = labse_score
      
      experiment.log_metrics(to_log, epoch=epoch+1)
      

Ovaa f-ja e istata kako prethodnata, samo shto raboti malku poinaku t.e. koristi gradient accumulation za da se ima pogolem effective batch size. Razlikata e vo toa koga se pravi zero_grad() i step() na optimizerot.

In [ ]:
def train_grad_acc(model, dataloader, experiment, hyperparameters, tokenized_labels, do_logging=True, validation_dataloader=None):

  optimizer = torch.optim.AdamW(model.parameters(), lr=hyperparameters['lr_rate'])# lr=0.0005)

  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  model.to(device)

  num_training_steps = hyperparameters['num_epochs'] * len(dataloader)
  effective_num_training_steps = hyperparameters['num_epochs'] * (len(dataloader)/hyperparameters['num_grad_acc_steps'])

  #lr_scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=int(0.05*num_training_steps), num_training_steps=effective_num_training_steps)
  lr_scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=int(hyperparameters['num_warmup_steps_percent_of_all']*effective_num_training_steps))

  progress_bar = tqdm(range(num_training_steps))

  global_step = 0

  experiment.log_parameters(hyperparameters)

  model.train()
  for epoch in range(hyperparameters['num_epochs']):
    running_avg = 0.0
    running_avg_loss = 0.0
    epoch_step = 0

    if epoch == 0 and validation_dataloader is not None:
      # do a validation run before training
      rouge_score, sacrebleu_score, bertscore_score, labse_score, _ = evaluate(model, validation_dataloader, device, None, untokenized_labels)

      to_log = get_metrics_dict(rouge_score, sacrebleu_score, bertscore_score)
      to_log['labse_score'] = labse_score
      
      experiment.log_metrics(to_log, epoch=epoch)

    for batch_num, batch in enumerate(dataloader):
      batch = {k: v.to(device) for k, v in batch.items()}

      outputs = model(**batch)
      loss = outputs.loss / hyperparameters['num_grad_acc_steps']
      loss.backward()

      if ((batch_num+1) % hyperparameters['num_grad_acc_steps']) == 0 or batch_num+1 == len(dataloader):
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

      progress_bar.update(1)

      if not do_logging:
        del batch
        continue

      preds = torch.argmax(outputs.logits.detach(), dim=-1)
      labels_np = batch['labels'].detach()

      num_correct = (labels_np[labels_np != -100] == preds[labels_np != -100]).sum().item()
      num_total = labels_np[labels_np != -100].numel()

      del batch
      del preds
      del labels_np

      #num_correct = (preds == batch['labels'] and batch['labels'] != -100).sum().item()
      #num_total = batch['labels'][batch['labels'] != -100].numel()

      batch_acc = (float(num_correct)/num_total)
      
      epoch_step += 1
      global_step += 1

      the_loss = loss.item()
      
      running_avg = ((epoch_step-1)/float(epoch_step))*running_avg + batch_acc/float(epoch_step)
      running_avg_loss = ((epoch_step-1)/float(epoch_step))*running_avg_loss + the_loss/float(epoch_step)

      progress_bar.set_postfix_str(f"batch_loss={the_loss:.5f} epoch_running_avg_loss={running_avg_loss:.5f} batch_acc={batch_acc:.5f}\tepoch_running_avg_acc={running_avg:.5f}", refresh=True)

      experiment.log_metric('batch_acc', batch_acc, step=global_step)
      experiment.log_metric(f'batch_acc_epoch_{epoch}', batch_acc, step=epoch_step)

      experiment.log_metric('batch_loss', the_loss, step=global_step)
      
      experiment.log_metric('per_epoch_running_avg_acc', running_avg, step=global_step)
      experiment.log_metric('per_epoch_running_avg_loss', running_avg_loss, step=global_step)
      
    if validation_dataloader is not None:
      rouge_score, sacrebleu_score, bertscore_score, labse_score, _ = evaluate(model, validation_dataloader, device, None, untokenized_labels)

      to_log = get_metrics_dict(rouge_score, sacrebleu_score, bertscore_score)
      to_log['labse_score'] = labse_score
      
      experiment.log_metrics(to_log, epoch=epoch+1)
      

Ovaa funkcija go presmetuva labse skorot, ovoj kod mi e kopiran od internet.

In [ ]:
def calculate_labSE_score(outputs, labels):
  
  def normalization(embeds):
    norms = np.linalg.norm(embeds, 2, axis=1, keepdims=True)
    return embeds/norms

  preprocessor = hub.KerasLayer(
      "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
  encoder = hub.KerasLayer("https://tfhub.dev/google/LaBSE/2")

  dataset = DatasetFromDictData({'outputs': outputs, 'labels': labels})
  dataloader = torch.utils.data.DataLoader(dataset, batch_size=8)

  similarities = []

  for batch in dataloader:
    predictions = tf.constant(batch['outputs'])
    references = tf.constant(batch['labels'])

    predictions_embeds = encoder(preprocessor(predictions))["default"]
    references_embeds = encoder(preprocessor(references))["default"]

    # For semantic similarity tasks, apply l2 normalization to embeddings
    predictions_embeds = normalization(predictions_embeds)
    references_embeds = normalization(references_embeds)

    similarities.extend(np.matmul(predictions_embeds, np.transpose(references_embeds)).diagonal().tolist())

  return np.mean(similarities)

Ovaa f-ja gi generira izlezite od modelot za test mnozhestvoto i gi priprema labelite za koristenje vo f-jata evaluate.

In [ ]:
def get_preds_and_labels_for_eval_test(model, evaluation_dataloader, device):
  decoded_outputs = []
  tokenized_outputs = []

  eval_progress_bar = tqdm(range(len(evaluation_dataloader)))

  with torch.no_grad():
    model.eval()
    for batch in evaluation_dataloader:
      batch = {k: v.to(device) for k, v in batch.items()}

      curr_outp = model.generate(**batch, max_length=300, num_beams=3, early_stopping=True)

      del batch

      decoded_output = tokenizer.batch_decode(curr_outp.tolist(), skip_special_tokens=True)
      
      # dirty trick to get rid of the special tokens and have the raw tokens instead of decoded output
      tokenized_output = tokenizer(decoded_output, add_special_tokens=False)['input_ids']

      decoded_outputs.extend(decoded_output)
      tokenized_outputs.extend(tokenized_output)

      eval_progress_bar.update(1)

  return decoded_outputs, tokenized_outputs

Ovaa f-ja ja pravi evaluacijata na modelot t.e. gi presmetuva site skorovi(bez rouge-we) i gi vrakja.

In [ ]:
def evaluate(model, evaluation_dataloader, device, tokenized_labels, untokenized_labels, untokenized_inputs=None):

  decoded_outputs, tokenized_outputs = get_preds_and_labels_for_eval_test(model, evaluation_dataloader, device)
  references_for_sacrebleu = None
    
  # Dokolku ima povekje od 1500 primeroci za testiranje, se namaluvaat do 1500 za
  # testiranjeto da zavrshi vo razumno vreme. Toa se pravi so random sampling,
  # taka shto se zemaat 1500 random indeksi vo dozvoleniot range i se zemaat
  # primerocite so tie indeksi
  if len(tokenized_labels) > 1500:
    print(f'{len(tokenized_labels)} samples, reducing down to 1500')
  do = np.array(decoded_outputs, dtype=object)
  to = np.array(tokenized_outputs, dtype=object)
  tl = np.array(tokenized_labels, dtype=object)
  ul = np.array(untokenized_labels, dtype=object)
  ui = np.array(untokenized_inputs, dtype=object)

  indicies = list(range(0, len(tokenized_labels)))
  np.random.shuffle(indicies)

  indicies = indicies[0:1499]

  decoded_outputs = do[indicies].tolist()
  tokenized_outputs = to[indicies].tolist()
  tokenized_labels = tl[indicies].tolist()
  untokenized_labels = ul[indicies].tolist()
  untokenized_inputs = ui[indicies].tolist()

  label_references_for_sacrebleu = [[lab] for lab in untokenized_labels]
  input_references_for_sacrebleu = [[inp] for inp in untokenized_inputs]

  print(f'new len:{len(tokenized_labels)}')
    
  rouge = load_metric('rouge')
  sacrebleu = load_metric('sacrebleu')
  bertscore = load_metric('bertscore')

  label_output_rouge_score = rouge.compute(predictions=decoded_outputs, references=untokenized_labels, rouge_types=['rouge1', 'rouge2', 'rouge3', 'rouge4', 'rougeL', 'rougeLsum'])
  print('rouge1 computed')
  label_output_sacrebleu_score = sacrebleu.compute(predictions=decoded_outputs, references=label_references_for_sacrebleu)
  print('sacrebleu1 computed')
  label_output_bertscore_score = bertscore.compute(predictions=decoded_outputs, references=untokenized_labels, lang='en')
  print('bertscore1 computed')
  label_output_labse_score = calculate_labSE_score(decoded_outputs, untokenized_labels)
  print('labsescore1 computed')
    
  if untokenized_inputs is not None:
    print('vleze')
    references_for_sacrebleu = [[lab] for lab in untokenized_inputs]

    input_output_rouge_score = rouge.compute(predictions=decoded_outputs, references=untokenized_inputs, rouge_types=['rouge1', 'rouge2', 'rouge3', 'rouge4', 'rougeL', 'rougeLsum'])
    print('rouge2 computed')
    input_output_sacrebleu_score = sacrebleu.compute(predictions=decoded_outputs, references=input_references_for_sacrebleu)
    print('sacrebleu2 computed')
    input_output_bertscore_score = bertscore.compute(predictions=decoded_outputs, references=untokenized_inputs, lang='en')
    print('bertscore2 computed')
    input_output_labse_score = calculate_labSE_score(decoded_outputs, untokenized_inputs)
    print('labsescore2 computed')
    
    return label_output_rouge_score, label_output_sacrebleu_score, label_output_bertscore_score, label_output_labse_score, input_output_rouge_score, input_output_sacrebleu_score, input_output_bertscore_score, input_output_labse_score, decoded_outputs
  else:
    print('else')
    return label_output_rouge_score, label_output_sacrebleu_score, label_output_bertscore_score, label_output_labse_score, decoded_outputs

In [ ]:
def evaluate_directly(device, untokenized_labels, untokenized_inputs, untokenized_outputs):

  label_references_for_sacrebleu = [[lab] for lab in untokenized_labels]
  input_references_for_sacrebleu = [[inp] for inp in untokenized_inputs]

  print(f'new len:{len(untokenized_labels)}')
    
  rouge = load_metric('rouge')
  sacrebleu = load_metric('sacrebleu')
  bertscore = load_metric('bertscore')

  label_output_rouge_score = rouge.compute(predictions=untokenized_outputs, references=untokenized_labels, rouge_types=['rouge1', 'rouge2', 'rouge3', 'rouge4', 'rougeL', 'rougeLsum'])
  print('rouge1 computed')
  label_output_sacrebleu_score = sacrebleu.compute(predictions=untokenized_outputs, references=label_references_for_sacrebleu)
  print('sacrebleu1 computed')
  label_output_bertscore_score = bertscore.compute(predictions=untokenized_outputs, references=untokenized_labels, lang='en')
  print('bertscore1 computed')
  label_output_labse_score = calculate_labSE_score(untokenized_outputs, untokenized_labels)
  print('labsescore1 computed')
    
  if untokenized_inputs is not None:
    print('vleze')
    references_for_sacrebleu = [[lab] for lab in untokenized_inputs]

    input_output_rouge_score = rouge.compute(predictions=untokenized_outputs, references=untokenized_inputs, rouge_types=['rouge1', 'rouge2', 'rouge3', 'rouge4', 'rougeL', 'rougeLsum'])
    print('rouge2 computed')
    input_output_sacrebleu_score = sacrebleu.compute(predictions=untokenized_outputs, references=input_references_for_sacrebleu)
    print('sacrebleu2 computed')
    input_output_bertscore_score = bertscore.compute(predictions=untokenized_outputs, references=untokenized_inputs, lang='en')
    print('bertscore2 computed')
    input_output_labse_score = calculate_labSE_score(untokenized_outputs, untokenized_inputs)
    print('labsescore2 computed')
    
    return label_output_rouge_score, label_output_sacrebleu_score, label_output_bertscore_score, label_output_labse_score, input_output_rouge_score, input_output_sacrebleu_score, input_output_bertscore_score, input_output_labse_score
  else:
    print('else')
    return label_output_rouge_score, label_output_sacrebleu_score, label_output_bertscore_score, label_output_labse_score

Ovaa funkcija samo gi reorganizira podatocite, t.e. dobiva kako vlez najrazlichni skorovi, gi sumira i gi pakuva kako eden rechnik koj e pregleden i se prikachuva na comet.

In [ ]:
def get_metrics_dict(label_output_rouge_score, label_output_sacrebleu_score, label_output_bertscore_score, label_output_labse_score,
                     input_output_rouge_score, input_output_sacrebleu_score, input_output_bertscore_score, input_output_labse_score):
  metrics = {}
  for rouge_score_type, rouge_score in zip(['input_output', 'label_output'], [input_output_rouge_score, label_output_rouge_score]):
    for suffix in ['1', '2', '3', '4', 'L', 'Lsum']:
      metrics[f'{rouge_score_type}_rouge{suffix}_precision'] = rouge_score[f'rouge{suffix}'].mid.precision
      metrics[f'{rouge_score_type}_rouge{suffix}_recall'] = rouge_score[f'rouge{suffix}'].mid.recall
      metrics[f'{rouge_score_type}_rouge{suffix}_fmeasure'] = rouge_score[f'rouge{suffix}'].mid.fmeasure

  metrics['input_output_sacrebleu_score'] = input_output_sacrebleu_score['score']
  metrics['label_output_sacrebleu_score'] = label_output_sacrebleu_score['score']

  metrics['input_output_bertscore_f1'] = np.mean(input_output_bertscore_score['f1'])
  metrics['input_output_bertscore_precision'] = np.mean(input_output_bertscore_score['precision'])
  metrics['input_output_bertscore_recall'] = np.mean(input_output_bertscore_score['recall'])

  metrics['label_output_bertscore_f1'] = np.mean(label_output_bertscore_score['f1'])
  metrics['label_output_bertscore_precision'] = np.mean(label_output_bertscore_score['precision'])
  metrics['label_output_bertscore_recall'] = np.mean(label_output_bertscore_score['recall'])

  return metrics

In [ ]:
import os

if not os.path.isdir('/content/model_data_training'):
    os.mkdir('/content/model_data_training')

if not os.path.isdir('/content/trained_models'):
    os.mkdir('/content/trained_models')

Ova e kodot so koj go generirav primerokot od nlu nt podatochnoto mnozhestvo. Sega e iskomentiran bidejkji mnozhestvoto e kreirano i zachuvano, ne e potreben povekje.

In [ ]:
# save the random sample from nlu_np dataset so that all the models use the same sample
# dataset1_train = read_dataset_nlu_nt('../input/all-data/Datasets za praksa-20220309T182614Z-001/Datasets za praksa/nlu_nt_dataset/sent2sent_train.tsv')
# dataset1_val = read_dataset_nlu_nt('../input/all-data/Datasets za praksa-20220309T182614Z-001/Datasets za praksa/nlu_nt_dataset/sent2sent_test.tsv')

# dataset1_train = dataset1_train.sample(frac=0.0625)
# dataset1_val = dataset1_val.sample(frac=0.25)

# import os

# os.mkdir('./nlu_nt_dataset sample2')
# dataset1_train.to_csv('./nlu_nt_dataset sample2/sent2sent_train.tsv', sep='\t')
# dataset1_val.to_csv('./nlu_nt_dataset sample2/sent2sent_test.tsv', sep='\t')

In [ ]:
# create the combined dataset so that all the models use the same combination/dataset

#dataset_msr_train = read_dataset1('/content/datasets/msr_paraphrase_train.txt')
#dataset_msr_test = read_dataset1('/content/datasets/msr_paraphrase_test.txt')

#dataset_nlu_nt_train = read_dataset_nlu_nt_sample('/content/datasets/nlu_nt_train.tsv')
#dataset_nlu_nt_test = read_dataset_nlu_nt_sample('/content/datasets/nlu_nt_test.tsv')

#dataset_paws_train = read_dataset_paws('/content/datasets/paws_wiki_train.tsv')
#dataset_paws_test = read_dataset_paws('/content/datasets/paws_wiki_test.tsv')

#dataset_msr_train = dataset_msr_train.sample(frac=0.30)
#dataset_msr_test = dataset_msr_test.sample(frac=0.30)

#dataset_nlu_nt_train = dataset_nlu_nt_train.sample(frac=0.30)
#dataset_nlu_nt_test = dataset_nlu_nt_test.sample(frac=0.30)

#dataset_paws_train = dataset_paws_train.sample(frac=0.30)
#dataset_paws_test = dataset_paws_test.sample(frac=0.30)

#combined_dataset_train = dataset_msr_train[['#1 String', '#2 String']].append([dataset_nlu_nt_train[['#1 String', '#2 String']],
 #                                                                              dataset_paws_train[['#1 String', '#2 String']]], ignore_index=True)

#combined_dataset_test = dataset_msr_test[['#1 String', '#2 String']].append([dataset_nlu_nt_test[['#1 String', '#2 String']],
#                                                                               dataset_paws_test[['#1 String', '#2 String']]], ignore_index=True)

#combined_dataset_train.to_csv('/content/drive/MyDrive/Praksa/Datasets za praksa/combined_dataset/combined_train.csv')
#combined_dataset_test.to_csv('/content/drive/MyDrive/Praksa/Datasets za praksa/combined_dataset/combined_test.csv')

#print(combined_dataset_train.shape)
#print(combined_dataset_test.shape)

# Rouge-WE setup

Go simnuva toj golemiot fajl so embeddinzi.

In [ ]:
import gdown
gdown.download(url='https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
To: /content/GoogleNews-vectors-negative300.bin.gz
100%|██████████| 1.65G/1.65G [00:31<00:00, 53.0MB/s]


'GoogleNews-vectors-negative300.bin.gz'

Simnuva datoteka koja gi sodrzhi potrebnite raboti za presmetka na rouge-we, kako na pr. serverot, perl skriptata za rouge-we...

In [ ]:
gdown.download(url='https://drive.google.com/uc?id=1T7EzXJBD0OkDKmGimmLM7yM06r3m7oJT', output='rouge-we.rar', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1T7EzXJBD0OkDKmGimmLM7yM06r3m7oJT
To: /content/rouge-we.rar
100%|██████████| 2.63M/2.63M [00:00<00:00, 129MB/s]


'rouge-we.rar'

In [ ]:
!unrar x /content/rouge-we.rar /content/rouge-we/


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/rouge-we.rar

Creating    /content/rouge-we                                         OK
Extracting  /content/rouge-we/final_evaluation_script.py                   0%  OK 
Extracting  /content/rouge-we/LICENSE                                      0%  OK 
Extracting  /content/rouge-we/README.md                                    0%  OK 
Extracting  /content/rouge-we/ROUGE-WE-1.0.0.pl                            0%  OK 
Extracting  /content/rouge-we/sample-config.xml                            0%  OK 
Extracting  /content/rouge-we/trial_evaluation_script.py                   0%  OK 
Extracting  /content/rouge-we/word2vec_server.m.py                         1%  OK 
Creating    /content/rouge-we/data_google_drive                       OK
Extracting  /content/rouge-we/data_google_drive/sshleifer_distilbart-cnn-6-6 1.0.pickle       8%

Slednite nekolku komandi mora da se izvrshat za da se instaliraat potrebnite raboti za da mozhe da raboti perl skriptata za presmetka na rouge-we.

In [ ]:
!perl -MCPAN -e 'install Bundle::LWP'


CPAN.pm requires configuration, but most of it can be done automatically.
If you answer 'no' below, you will enter an interactive dialog for each
configuration option instead.

Would you like to configure as much as possible automatically? [yes] y


Autoconfiguration complete.

commit: wrote '/root/.cpan/CPAN/MyConfig.pm'

You can re-run configuration any time with 'o conf init' in the CPAN shell
Fetching with HTTP::Tiny:
http://www.cpan.org/authors/01mailrc.txt.gz
Reading '/root/.cpan/sources/authors/01mailrc.txt.gz'
............................................................................DONE
Fetching with HTTP::Tiny:
http://www.cpan.org/modules/02packages.details.txt.gz
Reading '/root/.cpan/sources/modules/02packages.details.txt.gz'
  Database was generated on Sun, 04 Sep 2022 17:17:03 GMT
  HTTP::Date not available
..............
  New CPAN.pm version (v2.34) available.
  [Currently running version is v2.18]
  You might want to try
    install CPAN
    reload cpan
  to both upg

In [ ]:
!sudo cpan JSON

Loading internal null logger. Install Log::Log4perl for logging messages
Reading '/root/.cpan/Metadata'
  Database was generated on Sun, 04 Sep 2022 17:17:03 GMT
Running install for module 'JSON'
Fetching with LWP:
http://www.cpan.org/authors/id/I/IS/ISHIGAKI/JSON-4.09.tar.gz
Fetching with LWP:
http://www.cpan.org/authors/id/I/IS/ISHIGAKI/CHECKSUMS
Checksum for /root/.cpan/sources/authors/id/I/IS/ISHIGAKI/JSON-4.09.tar.gz ok
'YAML' not installed, will not store persistent state
Configuring I/IS/ISHIGAKI/JSON-4.09.tar.gz with Makefile.PL
Welcome to JSON (v.4.09)

 ** BACKWARD INCOMPATIBILITY **

Since version 2.90, stringification (and string comparison) for
JSON::true and JSON::false has not been overloaded. It shouldn't
matter as long as you treat them as boolean values, but a code that
expects they are stringified as "true" or "false" doesn't work as
you have expected any more.

    if (JSON::true eq 'true') {  # now fails

    print "The result is 1 now."; # => The result is 1 now.


In [ ]:
!perl -MCPAN -e 'install XML::DOM'

Reading '/root/.cpan/Metadata'
  Database was generated on Sun, 04 Sep 2022 17:17:03 GMT
Running install for module 'XML::DOM'
Fetching with LWP:
http://www.cpan.org/authors/id/T/TJ/TJMATHER/XML-DOM-1.46.tar.gz
Fetching with LWP:
http://www.cpan.org/authors/id/T/TJ/TJMATHER/CHECKSUMS
Checksum for /root/.cpan/sources/authors/id/T/TJ/TJMATHER/XML-DOM-1.46.tar.gz ok
'YAML' not installed, will not store persistent state
Configuring T/TJ/TJMATHER/XML-DOM-1.46.tar.gz with Makefile.PL
Checking if your kit is complete...
Looks good
Generating a Unix-style Makefile
Writing Makefile for XML::DOM
Writing MYMETA.yml and MYMETA.json
  TJMATHER/XML-DOM-1.46.tar.gz
  /usr/bin/perl Makefile.PL INSTALLDIRS=site -- OK
Running make for T/TJ/TJMATHER/XML-DOM-1.46.tar.gz
---- Unsatisfied dependencies detected during ----
----       TJMATHER/XML-DOM-1.46.tar.gz       ----
    XML::Parser [requires]
    XML::Parser::PerlSAX [requires]
    XML::RegExp [requires]
Running install for module 'XML::Parser'
Fetchi

Slednite dve kelii go brishat fajlot WordNet-2.0.exc.db i odnovo go generiraat, bidejkji inaku javuva problem, ne znam zoshto, no ova reshenie go najdov na internet.

In [ ]:
!rm '/content/rouge-we/rouge_1.5.5_data/WordNet-2.0.exc.db'

In [ ]:
!perl '/content/rouge-we/rouge_1.5.5_data/WordNet-2.0-Exceptions/buildExeptionDB.pl' '/content/rouge-we/rouge_1.5.5_data/WordNet-2.0-Exceptions' '/content/rouge-we/rouge_1.5.5_data/smart_common_words.txt' '/content/rouge-we/rouge_1.5.5_data/WordNet-2.0.exc.db'

Ovoj kod go najdov na internet(originalno beshe za rouge) i go modificirav da raboti za rouge-we.

In [ ]:
import pickle
import subprocess
import random
from comet_ml import ExistingExperiment

# GOTTEN FROM INTERNET
import os
import re
import shutil
import subprocess
import sys
from glob import glob
from tempfile import mkdtemp
from typing import Dict, List, Optional
from os import makedirs

def _write_summaries(summaries, references, hyp_dir, ref_dir):
    # type: (List[str], List[List[str]], str, str) -> None
    makedirs(hyp_dir, exist_ok=True)
    makedirs(ref_dir, exist_ok=True)

    for i, hyp in enumerate(summaries):
        hyp_path = os.path.join(hyp_dir, '{}.txt'.format(i))
        with open(hyp_path, 'w', encoding="utf-8") as f:
            f.write(hyp)

    for i, multi_ref in enumerate(references):
        for j, ref in enumerate(multi_ref):
            ref_path = os.path.join(ref_dir, '{}.{}.txt'.format(i, j))
            with open(ref_path, 'w', encoding="utf-8") as f:
                f.write(ref)


def _write_config(config_path, hyp_dir, ref_dir):
    # type: (str, str, str) -> None
    xml = '<ROUGE-EVAL version="1.5.5">'
    for n, hyp in enumerate(glob(os.path.join(hyp_dir, '*'))):
        if not os.path.isfile(hyp):
            continue
        hyp_name = os.path.basename(hyp)
        hyp_stem, _ = os.path.splitext(hyp_name)

        hyp_str = '<P ID="{}">{}</P>'.format('A', hyp_name)

        ref_paths = glob(os.path.join(ref_dir, '{}.*'.format(hyp_stem)))
        ref_paths = [ref for ref in ref_paths if os.path.isfile(ref)]
        if not ref_paths:
            raise RuntimeError('Reference not found for {}'.format(hyp))
        ref_str = '\n'.join(
            '<M ID="{}">{}</M>'.format(idx, os.path.basename(ref))
            for idx, ref in enumerate(ref_paths))

        xml += """
<EVAL ID="{eval_id}">
<MODEL-ROOT>{model_root}</MODEL-ROOT>
<PEER-ROOT>{peer_root}</PEER-ROOT>
<INPUT-FORMAT TYPE="SPL">
</INPUT-FORMAT>
<PEERS>
    {peers}
</PEERS>
<MODELS>
    {models}
</MODELS>
</EVAL>""".format(eval_id=n + 1, model_root=ref_dir, peer_root=hyp_dir,
              peers=hyp_str, models=ref_str)
    xml += '</ROUGE-EVAL>'

    with open(config_path, 'w') as f:
        f.write(xml)

# END OF GOTTEN FROM INTERNET

def get_results(prefix=""):
	rouge_call = subprocess.Popen(["perl", "/content/rouge-we/ROUGE-WE-1.0.0.pl", "-n", "4", "-U", "-2", "4", "-e",
		"/content/rouge-we/rouge_1.5.5_data/", "-c", "95", "-a", "/content/rouge-we/sample-config.xml"], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

	output, errors = rouge_call.communicate()
	rouge_call.wait()
 
	print(f'output:{output}')
	print(f'errors:{errors}')

	tokens = output.split()

	results = {}

	for idx, token in enumerate(tokens):
		if "ROUGE" in token:
			what_rouge = token

			if idx + 2 < len(tokens):
				metric = tokens[idx + 1][-2]
				value = tokens[idx + 2]
				results[f'test_{prefix}_WE_{what_rouge}_{metrics_long[metric]}'.lower()] = float(value)

	return results

def delete_all_files_in_dir(dir_path):
	for f in glob(f'{dir_path}*'):
		os.remove(f)

# Experiments

Tuka se definiraat site eksperimenti. Dokolku e potrebno da se izvrshat samo nekoi, drugite mozhe da se iskomentiraat.

In [ ]:
experiments = {
    'EleutherAI/gpt-neo-125M 4.0': {
        'experiment_name': 'EleutherAI/gpt-neo-125M 4.0',
        'model_name': 'EleutherAI/gpt-neo-125M',
        'hyperparameters': {
            'num_epochs': 1,
            'batch_size': 8,
            'optimizer': 'AdamW',
            'dataset': 'msr_paraphrase',
            'lr_rate': 0.00001,
            'num_warmup_steps_percent_of_all': 0.1,
            'lr_schedule': 'get_constant_schedule_with_warmup'
        },
        'read_dataset_function' : read_dataset1,
        'dataset_path_train': '/content/datasets/msr_paraphrase_train.txt',
        'dataset_path_test': '/content/datasets/msr_paraphrase_test.txt',
        'dataset_preprocessing_function': preprocess_for_gpt_neo,
        'prepare_and_tokenize_dataset_function': prepare_and_tokenize_dataset_gpt_neo,
        'prepare_and_tokenize_test_dataset_function': prepare_and_tokenize_test_dataset_gpt_neo,
        'training_function': train,
        'tokenizer_class': GPT2Tokenizer,
        'model_class': GPTNeoForCausalLM
    },
}

Se izvrshuvaat eksperimentite eden po eden.

In [ ]:
for experiment_name, experiment_data in experiments.items():
  comet = Experiment("IfeqLAdsfDx5g915IE4TeV69t", project_name='Praksa Experiments probi')
  comet.set_name(experiment_name)

  comet.log_parameters(experiment_data['hyperparameters'])

  tokenizer = experiment_data['tokenizer_class'].from_pretrained(experiment_data['model_name'])
  model = experiment_data['model_class'].from_pretrained(experiment_data['model_name'])

  dataset_train = experiment_data['read_dataset_function'](experiment_data['dataset_path_train'])
  dataset_test = experiment_data['read_dataset_function'](experiment_data['dataset_path_test'])

  if experiment_data['dataset_preprocessing_function'] is not None:
    # Se preprocesiraat podatocite dokolku e potrebno so soodvetnata f-ja definirana vo eksperimentot
    experiment_data['dataset_preprocessing_function'](dataset_train)
    experiment_data['dataset_preprocessing_function'](dataset_test)

  data_train = experiment_data['prepare_and_tokenize_dataset_function'](dataset_train, tokenizer, model)
  data_val, tokenized_labels, untokenized_labels, untokenized_input = experiment_data['prepare_and_tokenize_test_dataset_function'](dataset_test, tokenizer)

  train_dataset = DatasetFromDictData(data_train)
  train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=experiment_data['hyperparameters']['batch_size'], num_workers=2)

  val_dataset = DatasetFromDictData(data_val)
  val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=experiment_data['hyperparameters']['batch_size'], num_workers=2)

  # Se povikuva soodvetnata f-ja za treniranje spored opisot na eksperimentot
  experiment_data['training_function'](model, train_dataloader, comet, experiment_data['hyperparameters'], tokenized_labels, do_logging=True)
  torch.save(model.state_dict(), f'/content/trained_models/{comet.get_name().replace("/", "_")}.pt')

  # Se prazni RAM memorijata na GPU
  torch.cuda.empty_cache()

  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

  label_output_rouge_score, label_output_sacrebleu_score, label_output_bertscore_score, label_output_labse_score, input_output_rouge_score, input_output_sacrebleu_score, input_output_bertscore_score, input_output_labse_score, untokenized_output = evaluate(model, val_dataloader, device, tokenized_labels, untokenized_labels, untokenized_input)

  # Save data for later computation of RougeWE score
  dict_to_save = {
      'experiment_url': comet.url,
      'experiment_name': comet.get_name(),
      'experiment_key': comet.get_key(),
      'inputs_strings': untokenized_input,
      'model_outputs_strings': untokenized_output,
      'labels_strings': untokenized_labels
  }

  # Se zachuvuvaat generiranite rechenici, labelite i drugite potrebni podatoci za
  # presmetkata na rouge-we koja sleduva na krajot
  with open(f'/content/model_data_training/{comet.get_name().replace("/", "_")}.pickle', 'wb+') as handle:
      pickle.dump(dict_to_save, handle, protocol=pickle.HIGHEST_PROTOCOL)

  to_log = get_metrics_dict(label_output_rouge_score, label_output_sacrebleu_score, label_output_bertscore_score, label_output_labse_score,
  input_output_rouge_score, input_output_sacrebleu_score, input_output_bertscore_score, input_output_labse_score)

  to_log['label_output_labse_score'] = label_output_labse_score
  to_log['input_output_labse_score'] = input_output_labse_score

  comet.log_metrics(to_log, epoch=experiment_data['hyperparameters']['num_epochs'] + 1)

  for key in list(to_log.keys()):
    to_log[f'test_{key}'] = to_log.pop(key)

  comet.log_metrics(to_log)

  comet.log_others(to_log)

  comet.log_other('test_full_input_output_bleu_score', str(input_output_sacrebleu_score))
  comet.log_other('test_full_label_output_bleu_score', str(label_output_sacrebleu_score))

  comet.end()

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: Failed to calculate active processors count. Fall back to default CPU count 1
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. You can override where Comet is looking for a Git Patch by setting the configuration `COMET_GIT_DIRECTORY`
COMET INFO: Experiment is live on comet.com https://www.comet.com/stefandimeskistec/praksa-experiments-probi/e798248127d7474fa793a3fee93e0d2c

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3543: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transf

  0%|          | 0/510 [00:00<?, ?it/s]

  0%|          | 0/216 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

1725 samples, reducing down to 1500
new len:1499


...

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

bertscore1 computed
labsescore1 computed
vleze
rouge2 computed
sacrebleu2 computed
bertscore2 computed
labsescore2 computed


COMET WARNING: Couldn't retrieve Google Colab notebook content
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/stefandimeskistec/praksa-experiments-probi/e798248127d7474fa793a3fee93e0d2c
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     batch_acc [510]                       : (0.01568627450980392, 0.028639618138424822)
COMET INFO:     batch_acc_epoch_0 [510]               : (0.01568627450980392, 0.028639618138424822)
COMET INFO:     batch_loss [510]                      : (2.1780753135681152, 3.3382699489593506)
COMET INFO:     input_output_bertscore_f1             : 0.8960663266703636
COMET INFO:     input_output_bertscore_precision      : 0.8414244596523949
COMET INFO:     input_output_bertscore_recall         : 0.9631921853360691
COMET INFO:     input_output_labse_score             

# Paraphrasing via translation

In [ ]:
from deep_translator import GoogleTranslator, LibreTranslator

translation_experiments = {
     #'googletranslator msr': {
     #    'experiment_name': 'googletranslator msr',
     #    'read_dataset_function' : read_dataset1,
     #    'dataset_path_test': '/content/datasets/msr_paraphrase_test.txt',
     #    'translator_first': GoogleTranslator(source='en', target='de'),
     #    'translator_second': GoogleTranslator(source='de', target='en')
     #},

     #'googletranslator paws': {
     #    'experiment_name': 'googletranslator paws',
     #    'read_dataset_function' : read_dataset_paws,
     #    'dataset_path_test': '/content/datasets/paws_wiki_test.tsv',
     #    'translator_first': GoogleTranslator(source='en', target='de'),
     #    'translator_second': GoogleTranslator(source='de', target='en')
     #},

     #'googletranslator nlu_nt': {
     #    'experiment_name': 'googletranslator nlu_nt',
     #    'read_dataset_function' : read_dataset_nlu_nt_sample,
     #    'dataset_path_test': '/content/datasets/nlu_nt_test.tsv',
     #    'translator_first': GoogleTranslator(source='en', target='de'),
     #    'translator_second': GoogleTranslator(source='de', target='en')
     #},

      #'googletranslator combined': {
       #  'experiment_name': 'googletranslator combined',
        # 'read_dataset_function' : read_combined_dataset,
        # 'dataset_path_test': '/content/datasets/combined_dataset_test.csv',
        # 'translator_first': GoogleTranslator(source='en', target='de'),
        # 'translator_second': GoogleTranslator(source='de', target='en')
     #},

     'libretranslator msr': {
         'experiment_name': 'libretranslator msr',
         'read_dataset_function' : read_dataset1,
         'dataset_path_test': '/content/datasets/msr_paraphrase_test.txt',
         'translator_first': LibreTranslator(source='en', target='de'),
         'translator_second': LibreTranslator(source='de', target='en')
     },

     'libretranslator paws': {
         'experiment_name': 'libretranslator paws',
         'read_dataset_function' : read_dataset_paws,
         'dataset_path_test': '/content/datasets/paws_wiki_test.tsv',
         'translator_first': LibreTranslator(source='en', target='de'),
         'translator_second': LibreTranslator(source='de', target='en')
     },

     'libretranslator nlu_nt': {
         'experiment_name': 'libretranslator nlu_nt',
         'read_dataset_function' : read_dataset_nlu_nt_sample,
         'dataset_path_test': '/content/datasets/nlu_nt_test.tsv',
         'translator_first': LibreTranslator(source='en', target='de'),
         'translator_second': LibreTranslator(source='de', target='en')
     },

     'libretranslator combined': {
         'experiment_name': 'libretranslator combined',
         'read_dataset_function' : read_dataset_nlu_nt_sample,
         'dataset_path_test': '/content/datasets/nlu_nt_test.tsv',
         'translator_first': LibreTranslator(source='en', target='de'),
         'translator_second': LibreTranslator(source='de', target='en')
     }
}

In [ ]:
for experiment_name, experiment_data in translation_experiments.items():

  comet = Experiment("IfeqLAdsfDx5g915IE4TeV69t", project_name='Praksa Experiments final')
  comet.set_name(experiment_name)

  torch.cuda.empty_cache()

  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

  data = experiment_data['read_dataset_function'](experiment_data['dataset_path_test'])

  untokenized_inputs, untokenized_labels = data['#1 String'].tolist(), data['#2 String'].tolist()

  # Dokolku ima povekje od 750 primeroci za testiranje, se namaluvaat do 750 za
  # testiranjeto da zavrshi vo razumno vreme. Toa se pravi so random sampling,
  # taka shto se zemaat 750 random indeksi vo dozvoleniot range i se zemaat
  # primerocite so tie indeksi
  if len(untokenized_labels) > 750:
    print(f'{len(untokenized_labels)} samples, reducing down to 750')

    ul = np.array(untokenized_labels, dtype=object)
    ui = np.array(untokenized_inputs, dtype=object)

    indicies = list(range(0, len(untokenized_labels)))
    np.random.shuffle(indicies)

    indicies = indicies[0:749]

    untokenized_labels = ul[indicies].tolist()
    untokenized_inputs = ui[indicies].tolist()

  translated = experiment_data['translator_first'].translate_batch(untokenized_inputs)
  print(translated)
  untokenized_output = experiment_data['translator_second'].translate_batch(translated)

  label_output_rouge_score, label_output_sacrebleu_score, label_output_bertscore_score, label_output_labse_score, input_output_rouge_score, input_output_sacrebleu_score, input_output_bertscore_score, input_output_labse_score = evaluate_directly(device, untokenized_labels, untokenized_inputs, untokenized_output)

  # Save data for later computation of RougeWE score
  dict_to_save = {
      'experiment_url': comet.url,
      'experiment_name': comet.get_name(),
      'experiment_key': comet.get_key(),
      'inputs_strings': untokenized_inputs,
      'model_outputs_strings': untokenized_output,
      'labels_strings': untokenized_labels
  }

  # Se zachuvuvaat generiranite rechenici, labelite i drugite potrebni podatoci za
  # presmetkata na rouge-we koja sleduva na krajot
  with open(f'/content/model_data_training/{comet.get_name().replace("/", "_")}.pickle', 'wb+') as handle:
      pickle.dump(dict_to_save, handle, protocol=pickle.HIGHEST_PROTOCOL)

  to_log = get_metrics_dict(label_output_rouge_score, label_output_sacrebleu_score, label_output_bertscore_score, label_output_labse_score,
  input_output_rouge_score, input_output_sacrebleu_score, input_output_bertscore_score, input_output_labse_score)

  to_log['label_output_labse_score'] = label_output_labse_score
  to_log['input_output_labse_score'] = input_output_labse_score

  for key in list(to_log.keys()):
    to_log[f'test_{key}'] = to_log.pop(key)

  comet.log_metrics(to_log)

  comet.log_others(to_log)

  comet.log_other('test_full_input_output_bleu_score', str(input_output_sacrebleu_score))
  comet.log_other('test_full_label_output_bleu_score', str(label_output_sacrebleu_score))

  comet.end()

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/stefandimeskistec/praksa-experiments-final/9f4f8857cf9b4b8294926a9111a67013
COMET INFO:   Others:
COMET INFO:     Name         : mymemorytranslator msr
COMET INFO:     notebook_url : https://colab.research.google.com/notebook#fileId=1nPwcUPu9xvdd47HK8LrrFIXgPROKCAi7
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code        

1725 samples, reducing down to 750


ServerException: ignored

# Compute Rouge-WE for all experiments in /content/model_data_training/

Se pushta serverot za rouge-we da raboti vo pozadina za da ne ja blokira notebookot i za da mozhe da gi izvrshuva slednive kelii.

In [ ]:
!nohup python /content/rouge-we/word2vec_server.m.py -m /content/GoogleNews-vectors-negative300.bin.gz &

nohup: appending output to 'nohup.out'


Bidejkji raboti vo pozadina, izlezot mozhe edinstveno vo fajl da go zapishuva. Toj fajl e /content/nohup.out. Na serverot mu treba povekje vreme za da startuva (mozhebi ~3min?), pa ovaa kelija se izvrshuva se dodeka ne se startuva serverot, za da go blokira izvrshuvanjeto na slednata kelija duri da se pushti serverot.

In [ ]:
# wait until the server has started
# that will be indicated when tcmalloc:........
# is not the last line in the nohup.out file

import time

server_started = False
while server_started == False:
  with open("/content/nohup.out", "r") as file:
    if file.readlines()[-1].split(' ')[0] != 'tcmalloc:':
      server_started = True

    time.sleep(3)

Gi presmetuva rouge-we skorovite i gi loggira na comet.

In [ ]:
metrics_long = {'P': 'precision', 'R': 'recall', 'F': 'fmeasure'}

models_dir = '/content/rouge-we/sample-data/models/'
peers_dir = '/content/rouge-we/sample-data/peers/'
config_path = '/content/rouge-we/sample-config.xml'

for filename in glob('/content/model_data_training/*'):
  with open(filename, 'rb') as file:
    loaded_dict = pickle.load(file)

  experiment_key = loaded_dict['experiment_key']
  experiment_name = loaded_dict['experiment_name']
  inputs = loaded_dict['inputs_strings']
  outputs = loaded_dict['model_outputs_strings']
  labels = loaded_dict['labels_strings']

  print(f'Evaluating name={experiment_name} key={experiment_key}')

  # zema 100 random parovi (output od modelot, labela), bidejkji inaku e presporo
  to_shuffle = [(inp, lab, outp) for inp, lab, outp in zip(inputs, labels, outputs)]
  random.shuffle(to_shuffle)
  random_portion = to_shuffle[0:100]

  inputs = [inp for inp, lab, outp in random_portion]
  outputs = [outp for inp, lab, outp in random_portion]
  labels = [lab for inp, lab, outp in random_portion]

  inputs_prepared = [[inp] for inp in inputs]
  labels_prepared = [[label] for label in labels]

  delete_all_files_in_dir(models_dir)
  delete_all_files_in_dir(peers_dir)

  _write_summaries(summaries=outputs, references=inputs_prepared, hyp_dir=models_dir, ref_dir=peers_dir)
  _write_config(config_path=config_path, hyp_dir=models_dir, ref_dir=peers_dir)

  results_using_inputs = get_results(prefix='input_output')
  print("With input: ")
  print(results_using_inputs)

  delete_all_files_in_dir(models_dir)
  delete_all_files_in_dir(peers_dir)

  _write_summaries(summaries=outputs, references=labels_prepared, hyp_dir=models_dir, ref_dir=peers_dir)
  _write_config(config_path=config_path, hyp_dir=models_dir, ref_dir=peers_dir)

  results_using_labels = get_results(prefix='label_output')
  print("With labels: ")
  print(results_using_labels)

  exp = ExistingExperiment(api_key="IfeqLAdsfDx5g915IE4TeV69t", previous_experiment=experiment_key)
  exp.log_metrics(results_using_inputs)
  exp.log_metrics(results_using_labels)
  exp.log_others(results_using_inputs)
  exp.log_others(results_using_labels)
  exp.end()

Evaluating name=sshleifer/distill-pegasus-xsum-16-8 combined key=d35a056969b44bcbb24c3ab8dd6d428d
output:---------------------------------------------
A ROUGE-1 Average_R: 0.07884 (95%-conf.int. 0.06710 - 0.09113)
A ROUGE-1 Average_P: 0.07532 (95%-conf.int. 0.06585 - 0.08537)
A ROUGE-1 Average_F: 0.07311 (95%-conf.int. 0.06375 - 0.08262)
---------------------------------------------
A ROUGE-2 Average_R: 0.01128 (95%-conf.int. 0.00842 - 0.01425)
A ROUGE-2 Average_P: 0.01044 (95%-conf.int. 0.00783 - 0.01346)
A ROUGE-2 Average_F: 0.01076 (95%-conf.int. 0.00823 - 0.01336)
---------------------------------------------
A ROUGE-3 Average_R: 0.00331 (95%-conf.int. 0.00118 - 0.00567)
A ROUGE-3 Average_P: 0.00260 (95%-conf.int. 0.00103 - 0.00447)
A ROUGE-3 Average_F: 0.00388 (95%-conf.int. 0.00251 - 0.00553)
---------------------------------------------
A ROUGE-4 Average_R: 0.00218 (95%-conf.int. 0.00070 - 0.00368)
A ROUGE-4 Average_P: 0.00215 (95%-conf.int. 0.00081 - 0.00365)
A ROUGE-4 Average_

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


output:---------------------------------------------
A ROUGE-1 Average_R: 0.07075 (95%-conf.int. 0.06115 - 0.08116)
A ROUGE-1 Average_P: 0.07100 (95%-conf.int. 0.06266 - 0.07987)
A ROUGE-1 Average_F: 0.06740 (95%-conf.int. 0.05938 - 0.07591)
---------------------------------------------
A ROUGE-2 Average_R: 0.00943 (95%-conf.int. 0.00717 - 0.01196)
A ROUGE-2 Average_P: 0.00947 (95%-conf.int. 0.00727 - 0.01192)
A ROUGE-2 Average_F: 0.00918 (95%-conf.int. 0.00733 - 0.01141)
---------------------------------------------
A ROUGE-3 Average_R: 0.00277 (95%-conf.int. 0.00086 - 0.00474)
A ROUGE-3 Average_P: 0.00262 (95%-conf.int. 0.00144 - 0.00389)
A ROUGE-3 Average_F: 0.00339 (95%-conf.int. 0.00238 - 0.00449)
---------------------------------------------
A ROUGE-4 Average_R: -0.00217 (95%-conf.int. -0.00677 - 0.00118)
A ROUGE-4 Average_P: -0.00034 (95%-conf.int. -0.00192 - 0.00103)
A ROUGE-4 Average_F: 0.00150 (95%-conf.int. 0.00081 - 0.00228)
---------------------------------------------
A R

COMET ERROR: Failed to calculate active processors count. Fall back to default CPU count 1
COMET INFO: Experiment is live on comet.ml https://www.comet.com/stefandimeskistec/praksa-experiments-final/d35a056969b44bcbb24c3ab8dd6d428d

COMET INFO: -----------------------------------
COMET INFO: Comet.ml ExistingExperiment Summary
COMET INFO: -----------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/stefandimeskistec/praksa-experiments-final/d35a056969b44bcbb24c3ab8dd6d428d
COMET INFO:   Metrics:
COMET INFO:     test_input_output_we_rouge-1_fmeasure    : 0.07311
COMET INFO:     test_input_output_we_rouge-1_precision   : 0.07532
COMET INFO:     test_input_output_we_rouge-1_recall      : 0.07884
COMET INFO:     test_input_output_we_rouge-2_fmeasure    : 0.01076
COMET INFO:     test_input_output_we_rouge-2_precision   : 0.01044
COMET INFO:     test_input_output_we_rouge-2_recall      : 0.0112

Evaluating name=t5-base combined key=b536a4b53d874a66a60ac52b5d293a36
output:---------------------------------------------
A ROUGE-1 Average_R: 0.07211 (95%-conf.int. 0.06331 - 0.08129)
A ROUGE-1 Average_P: 0.07513 (95%-conf.int. 0.06545 - 0.08559)
A ROUGE-1 Average_F: 0.07053 (95%-conf.int. 0.06218 - 0.07935)
---------------------------------------------
A ROUGE-2 Average_R: 0.00674 (95%-conf.int. 0.00463 - 0.00910)
A ROUGE-2 Average_P: 0.00722 (95%-conf.int. 0.00492 - 0.00990)
A ROUGE-2 Average_F: 0.00695 (95%-conf.int. 0.00504 - 0.00908)
---------------------------------------------
A ROUGE-3 Average_R: 0.00178 (95%-conf.int. 0.00027 - 0.00333)
A ROUGE-3 Average_P: 0.00201 (95%-conf.int. 0.00047 - 0.00366)
A ROUGE-3 Average_F: 0.00315 (95%-conf.int. 0.00204 - 0.00439)
---------------------------------------------
A ROUGE-4 Average_R: 0.00131 (95%-conf.int. -0.00057 - 0.00327)
A ROUGE-4 Average_P: 0.00076 (95%-conf.int. -0.00071 - 0.00225)
A ROUGE-4 Average_F: 0.00205 (95%-conf.int. 

...

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: Failed to calculate active processors count. Fall back to default CPU count 1
COMET INFO: Experiment is live on comet.ml https://www.comet.com/stefandimeskistec/praksa-experiments-final/41f675889ab240d494ee66d7dc3452a8

COMET INFO: -----------------------------------
COMET INFO: Comet.ml ExistingExperiment Summary
COMET INFO: -----------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/stefandimeskistec/praksa-experiments-final/41f675889ab240d494ee66d7dc3452a8
COMET INFO:   Metrics:
COMET INFO:     test_input_output_we_rouge-1_fmeasure    : 0.5241
COMET INFO:     test_input_output_we_rouge-1_precision   : 0.5299
COMET INFO:     test_input_output_we_rouge-1_recall      : 0.51948
COMET INFO:     test_input_o

Evaluating name=googletranslator msr key=1f454d90384745acaa83989176ed37e0
output:---------------------------------------------
A ROUGE-1 Average_R: 0.49458 (95%-conf.int. 0.47966 - 0.50913)
A ROUGE-1 Average_P: 0.50375 (95%-conf.int. 0.49094 - 0.51584)
A ROUGE-1 Average_F: 0.49841 (95%-conf.int. 0.48490 - 0.51142)
---------------------------------------------
A ROUGE-2 Average_R: 0.39137 (95%-conf.int. 0.37098 - 0.41137)
A ROUGE-2 Average_P: 0.39854 (95%-conf.int. 0.37924 - 0.41866)
A ROUGE-2 Average_F: 0.39436 (95%-conf.int. 0.37453 - 0.41477)
---------------------------------------------
A ROUGE-3 Average_R: 0.33462 (95%-conf.int. 0.30973 - 0.36018)
A ROUGE-3 Average_P: 0.34038 (95%-conf.int. 0.31642 - 0.36607)
A ROUGE-3 Average_F: 0.33701 (95%-conf.int. 0.31328 - 0.36248)
---------------------------------------------
A ROUGE-4 Average_R: 0.29245 (95%-conf.int. 0.26542 - 0.32162)
A ROUGE-4 Average_P: 0.29781 (95%-conf.int. 0.27028 - 0.32709)
A ROUGE-4 Average_F: 0.29464 (95%-conf.int

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()


output:---------------------------------------------
A ROUGE-1 Average_R: 0.38027 (95%-conf.int. 0.36324 - 0.39607)
A ROUGE-1 Average_P: 0.37994 (95%-conf.int. 0.36182 - 0.39833)
A ROUGE-1 Average_F: 0.37495 (95%-conf.int. 0.35970 - 0.38946)
---------------------------------------------
A ROUGE-2 Average_R: 0.22450 (95%-conf.int. 0.20665 - 0.24231)
A ROUGE-2 Average_P: 0.22367 (95%-conf.int. 0.20455 - 0.24208)
A ROUGE-2 Average_F: 0.22069 (95%-conf.int. 0.20387 - 0.23777)
---------------------------------------------
A ROUGE-3 Average_R: 0.14947 (95%-conf.int. 0.12900 - 0.16928)
A ROUGE-3 Average_P: 0.14839 (95%-conf.int. 0.12820 - 0.16816)
A ROUGE-3 Average_F: 0.14653 (95%-conf.int. 0.12731 - 0.16469)
---------------------------------------------
A ROUGE-4 Average_R: 0.10438 (95%-conf.int. 0.08593 - 0.12340)
A ROUGE-4 Average_P: 0.10550 (95%-conf.int. 0.08618 - 0.12577)
A ROUGE-4 Average_F: 0.10283 (95%-conf.int. 0.08468 - 0.12174)
---------------------------------------------
A ROUGE

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: Failed to calculate active processors count. Fall back to default CPU count 1
COMET INFO: Experiment is live on comet.ml https://www.comet.com/stefandimeskistec/praksa-experiments-final/1f454d90384745acaa83989176ed37e0

COMET INFO: -----------------------------------
COMET INFO: Comet.ml ExistingExperiment Summary
COMET INFO: -----------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/stefandimeskistec/praksa-experiments-final/1f454d90384745acaa83989176ed37e0
COMET INFO:   Metrics:
COMET INFO:     test_input_output_we_rouge-1_fmeasure    : 0.49841
COMET INFO:     test_input_output_we_rouge-1_precision   : 0.50375
COMET INFO:     test_input_output_we_rouge-1_recall      : 0.49458
COMET INFO:     test_input